In [16]:
import pandas as pd
import glob
import os
import rasterio
import numpy as np

In [17]:
import tensorflow as tf
import numpy as np
import segmentation_models as sm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Segmentation Models: using `keras` framework.


In [19]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)
model_unet_500_5 = sm.Unet(BACKBONE, encoder_weights=None, input_shape=(128, 128, 2), classes=5, activation='softmax')


In [20]:
model_unet_500_5.compile(
    optimizer='Adam',
    loss=sm.losses.categorical_focal_jaccard_loss,
    metrics=[sm.metrics.iou_score, 'accuracy']
)

In [21]:
model_unet_500_5.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 128, 128, 2  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 128, 128, 2)  6           ['data[0][0]']                   
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 134, 134, 2)  0          ['bn_data[0][0]']                
                                                                                                  
 conv0 (Conv2D)                 (None, 64, 64, 64)   6272        ['zero_padding2d[0][0]']   

                                                                                                  
 stage1_unit3_bn2 (BatchNormali  (None, 32, 32, 64)  256         ['stage1_unit3_conv1[0][0]']     
 zation)                                                                                          
                                                                                                  
 stage1_unit3_relu2 (Activation  (None, 32, 32, 64)  0           ['stage1_unit3_bn2[0][0]']       
 )                                                                                                
                                                                                                  
 zero_padding2d_7 (ZeroPadding2  (None, 34, 34, 64)  0           ['stage1_unit3_relu2[0][0]']     
 D)                                                                                               
                                                                                                  
 stage1_un

                                                                                                  
 zero_padding2d_13 (ZeroPadding  (None, 18, 18, 128)  0          ['stage2_unit3_relu2[0][0]']     
 2D)                                                                                              
                                                                                                  
 stage2_unit3_conv2 (Conv2D)    (None, 16, 16, 128)  147456      ['zero_padding2d_13[0][0]']      
                                                                                                  
 add_5 (Add)                    (None, 16, 16, 128)  0           ['stage2_unit3_conv2[0][0]',     
                                                                  'add_4[0][0]']                  
                                                                                                  
 stage2_unit4_bn1 (BatchNormali  (None, 16, 16, 128)  512        ['add_5[0][0]']                  
 zation)  

 add_8 (Add)                    (None, 8, 8, 256)    0           ['stage3_unit2_conv2[0][0]',     
                                                                  'add_7[0][0]']                  
                                                                                                  
 stage3_unit3_bn1 (BatchNormali  (None, 8, 8, 256)   1024        ['add_8[0][0]']                  
 zation)                                                                                          
                                                                                                  
 stage3_unit3_relu1 (Activation  (None, 8, 8, 256)   0           ['stage3_unit3_bn1[0][0]']       
 )                                                                                                
                                                                                                  
 zero_padding2d_20 (ZeroPadding  (None, 10, 10, 256)  0          ['stage3_unit3_relu1[0][0]']     
 2D)      

                                                                                                  
 zero_padding2d_26 (ZeroPadding  (None, 10, 10, 256)  0          ['stage3_unit6_relu1[0][0]']     
 2D)                                                                                              
                                                                                                  
 stage3_unit6_conv1 (Conv2D)    (None, 8, 8, 256)    589824      ['zero_padding2d_26[0][0]']      
                                                                                                  
 stage3_unit6_bn2 (BatchNormali  (None, 8, 8, 256)   1024        ['stage3_unit6_conv1[0][0]']     
 zation)                                                                                          
                                                                                                  
 stage3_unit6_relu2 (Activation  (None, 8, 8, 256)   0           ['stage3_unit6_bn2[0][0]']       
 )        

 stage4_unit3_bn2 (BatchNormali  (None, 4, 4, 512)   2048        ['stage4_unit3_conv1[0][0]']     
 zation)                                                                                          
                                                                                                  
 stage4_unit3_relu2 (Activation  (None, 4, 4, 512)   0           ['stage4_unit3_bn2[0][0]']       
 )                                                                                                
                                                                                                  
 zero_padding2d_33 (ZeroPadding  (None, 6, 6, 512)   0           ['stage4_unit3_relu2[0][0]']     
 2D)                                                                                              
                                                                                                  
 stage4_unit3_conv2 (Conv2D)    (None, 4, 4, 512)    2359296     ['zero_padding2d_33[0][0]']      
          

                                                                                                  
 decoder_stage2b_relu (Activati  (None, 32, 32, 64)  0           ['decoder_stage2b_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 decoder_stage3_upsampling (UpS  (None, 64, 64, 64)  0           ['decoder_stage2b_relu[0][0]']   
 ampling2D)                                                                                       
                                                                                                  
 decoder_stage3_concat (Concate  (None, 64, 64, 128)  0          ['decoder_stage3_upsampling[0][0]
 nate)                                                           ',                               
                                                                  'relu0[0][0]']                  
          

In [23]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)
model_fpn_500_5 = sm.FPN(BACKBONE, encoder_weights=None, input_shape=(128, 128, 2), classes=5, activation='softmax')
# Callbacks
model_fpn_500_5.compile(
    optimizer='Adam',
    loss=sm.losses.categorical_focal_jaccard_loss,
    metrics=[sm.metrics.iou_score, 'accuracy']
)

In [24]:
model_fpn_500_5.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 128, 128, 2  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 128, 128, 2)  6           ['data[0][0]']                   
                                                                                                  
 zero_padding2d_34 (ZeroPadding  (None, 134, 134, 2)  0          ['bn_data[0][0]']                
 2D)                                                                                              
                                                                                            

 stage1_unit3_conv1 (Conv2D)    (None, 32, 32, 64)   36864       ['zero_padding2d_40[0][0]']      
                                                                                                  
 stage1_unit3_bn2 (BatchNormali  (None, 32, 32, 64)  256         ['stage1_unit3_conv1[0][0]']     
 zation)                                                                                          
                                                                                                  
 stage1_unit3_relu2 (Activation  (None, 32, 32, 64)  0           ['stage1_unit3_bn2[0][0]']       
 )                                                                                                
                                                                                                  
 zero_padding2d_41 (ZeroPadding  (None, 34, 34, 64)  0           ['stage1_unit3_relu2[0][0]']     
 2D)                                                                                              
          

 )                                                                                                
                                                                                                  
 zero_padding2d_47 (ZeroPadding  (None, 18, 18, 128)  0          ['stage2_unit3_relu2[0][0]']     
 2D)                                                                                              
                                                                                                  
 stage2_unit3_conv2 (Conv2D)    (None, 16, 16, 128)  147456      ['zero_padding2d_47[0][0]']      
                                                                                                  
 add_21 (Add)                   (None, 16, 16, 128)  0           ['stage2_unit3_conv2[0][0]',     
                                                                  'add_20[0][0]']                 
                                                                                                  
 stage2_un

                                                                                                  
 add_24 (Add)                   (None, 8, 8, 256)    0           ['stage3_unit2_conv2[0][0]',     
                                                                  'add_23[0][0]']                 
                                                                                                  
 stage3_unit3_bn1 (BatchNormali  (None, 8, 8, 256)   1024        ['add_24[0][0]']                 
 zation)                                                                                          
                                                                                                  
 stage3_unit3_relu1 (Activation  (None, 8, 8, 256)   0           ['stage3_unit3_bn1[0][0]']       
 )                                                                                                
                                                                                                  
 zero_padd

 )                                                                                                
                                                                                                  
 zero_padding2d_60 (ZeroPadding  (None, 10, 10, 256)  0          ['stage3_unit6_relu1[0][0]']     
 2D)                                                                                              
                                                                                                  
 stage3_unit6_conv1 (Conv2D)    (None, 8, 8, 256)    589824      ['zero_padding2d_60[0][0]']      
                                                                                                  
 stage3_unit6_bn2 (BatchNormali  (None, 8, 8, 256)   1024        ['stage3_unit6_conv1[0][0]']     
 zation)                                                                                          
                                                                                                  
 stage3_un

                                                                                                  
 stage4_unit3_bn2 (BatchNormali  (None, 4, 4, 512)   2048        ['stage4_unit3_conv1[0][0]']     
 zation)                                                                                          
                                                                                                  
 stage4_unit3_relu2 (Activation  (None, 4, 4, 512)   0           ['stage4_unit3_bn2[0][0]']       
 )                                                                                                
                                                                                                  
 zero_padding2d_67 (ZeroPadding  (None, 6, 6, 512)   0           ['stage4_unit3_relu2[0][0]']     
 2D)                                                                                              
                                                                                                  
 stage4_un

 segm_stage4b_conv (Conv2D)     (None, 16, 16, 128)  147456      ['segm_stage4a_relu[0][0]']      
                                                                                                  
 segm_stage5b_conv (Conv2D)     (None, 8, 8, 128)    147456      ['segm_stage5a_relu[0][0]']      
                                                                                                  
 segm_stage2b_conv (Conv2D)     (None, 64, 64, 128)  147456      ['segm_stage2a_relu[0][0]']      
                                                                                                  
 segm_stage3b_bn (BatchNormaliz  (None, 32, 32, 128)  512        ['segm_stage3b_conv[0][0]']      
 ation)                                                                                           
                                                                                                  
 segm_stage4b_bn (BatchNormaliz  (None, 16, 16, 128)  512        ['segm_stage4b_conv[0][0]']      
 ation)   

In [26]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)
model_Linknet_500_5 = sm.Linknet(BACKBONE, encoder_weights=None, input_shape=(128, 128, 2), classes=5, activation='softmax')
# Callbacks

In [27]:
model_Linknet_500_5.compile(
    optimizer='Adam',
    loss=sm.losses.categorical_focal_jaccard_loss,
    metrics=[sm.metrics.iou_score, 'accuracy']
)

In [28]:
model_Linknet_500_5.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 128, 128, 2  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 128, 128, 2)  6           ['data[0][0]']                   
                                                                                                  
 zero_padding2d_68 (ZeroPadding  (None, 134, 134, 2)  0          ['bn_data[0][0]']                
 2D)                                                                                              
                                                                                            

 stage1_unit3_conv1 (Conv2D)    (None, 32, 32, 64)   36864       ['zero_padding2d_74[0][0]']      
                                                                                                  
 stage1_unit3_bn2 (BatchNormali  (None, 32, 32, 64)  256         ['stage1_unit3_conv1[0][0]']     
 zation)                                                                                          
                                                                                                  
 stage1_unit3_relu2 (Activation  (None, 32, 32, 64)  0           ['stage1_unit3_bn2[0][0]']       
 )                                                                                                
                                                                                                  
 zero_padding2d_75 (ZeroPadding  (None, 34, 34, 64)  0           ['stage1_unit3_relu2[0][0]']     
 2D)                                                                                              
          

 )                                                                                                
                                                                                                  
 zero_padding2d_81 (ZeroPadding  (None, 18, 18, 128)  0          ['stage2_unit3_relu2[0][0]']     
 2D)                                                                                              
                                                                                                  
 stage2_unit3_conv2 (Conv2D)    (None, 16, 16, 128)  147456      ['zero_padding2d_81[0][0]']      
                                                                                                  
 add_37 (Add)                   (None, 16, 16, 128)  0           ['stage2_unit3_conv2[0][0]',     
                                                                  'add_36[0][0]']                 
                                                                                                  
 stage2_un

                                                                                                  
 add_40 (Add)                   (None, 8, 8, 256)    0           ['stage3_unit2_conv2[0][0]',     
                                                                  'add_39[0][0]']                 
                                                                                                  
 stage3_unit3_bn1 (BatchNormali  (None, 8, 8, 256)   1024        ['add_40[0][0]']                 
 zation)                                                                                          
                                                                                                  
 stage3_unit3_relu1 (Activation  (None, 8, 8, 256)   0           ['stage3_unit3_bn1[0][0]']       
 )                                                                                                
                                                                                                  
 zero_padd

 )                                                                                                
                                                                                                  
 zero_padding2d_94 (ZeroPadding  (None, 10, 10, 256)  0          ['stage3_unit6_relu1[0][0]']     
 2D)                                                                                              
                                                                                                  
 stage3_unit6_conv1 (Conv2D)    (None, 8, 8, 256)    589824      ['zero_padding2d_94[0][0]']      
                                                                                                  
 stage3_unit6_bn2 (BatchNormali  (None, 8, 8, 256)   1024        ['stage3_unit6_conv1[0][0]']     
 zation)                                                                                          
                                                                                                  
 stage3_un

                                                                                                  
 stage4_unit3_bn2 (BatchNormali  (None, 4, 4, 512)   2048        ['stage4_unit3_conv1[0][0]']     
 zation)                                                                                          
                                                                                                  
 stage4_unit3_relu2 (Activation  (None, 4, 4, 512)   0           ['stage4_unit3_bn2[0][0]']       
 )                                                                                                
                                                                                                  
 zero_padding2d_101 (ZeroPaddin  (None, 6, 6, 512)   0           ['stage4_unit3_relu2[0][0]']     
 g2D)                                                                                             
                                                                                                  
 stage4_un

 decoder_stage2a_bn (BatchNorma  (None, 16, 16, 32)  128         ['decoder_stage2a_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 decoder_stage2a_relu (Activati  (None, 16, 16, 32)  0           ['decoder_stage2a_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 decoder_stage2_upsampling (UpS  (None, 32, 32, 32)  0           ['decoder_stage2a_relu[0][0]']   
 ampling2D)                                                                                       
                                                                                                  
 decoder_stage2b_conv (Conv2D)  (None, 32, 32, 32)   9216        ['decoder_stage2_upsampling[0][0]
          

 decoder_stage4c_bn (BatchNorma  (None, 128, 128, 16  64         ['decoder_stage4c_conv[0][0]']   
 lization)                      )                                                                 
                                                                                                  
 decoder_stage4c_relu (Activati  (None, 128, 128, 16  0          ['decoder_stage4c_bn[0][0]']     
 on)                            )                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 5)  725         ['decoder_stage4c_relu[0][0]']   
                                                                                                  
 softmax (Activation)           (None, 128, 128, 5)  0           ['conv2d[0][0]']                 
                                                                                                  
Total para

In [29]:
import torch
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset
import segmentation_models_pytorch as smp
BACKBONE = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
INPUT_CHANNELS = 2
NUM_CLASSES = 5
model_pan_500_5 = smp.PAN(
    encoder_name=BACKBONE, 
    encoder_weights=ENCODER_WEIGHTS, 
    in_channels=INPUT_CHANNELS, 
    classes=NUM_CLASSES, 
    activation='softmax'
)

C:\Users\NNadi\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
model_pan_500_5.summary()

AttributeError: 'PAN' object has no attribute 'summary'

In [31]:
pip install torchinfo


Note: you may need to restart the kernel to use updated packages.


In [32]:
from torchinfo import summary

summary(model_pan_500_5, input_size=(1, 2, 128, 128))  # Example input size (batch_size, channels, height, width)


C:\Users\NNadi\anaconda3\envs\myenv\lib\site-packages\torch\nn\modules\module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Layer (type:depth-idx)                        Output Shape              Param #
PAN                                           [1, 5, 128, 128]          --
├─ResNetEncoder: 1-1                          [1, 2, 128, 128]          --
│    └─Conv2d: 2-1                            [1, 64, 64, 64]           6,272
│    └─BatchNorm2d: 2-2                       [1, 64, 64, 64]           128
│    └─ReLU: 2-3                              [1, 64, 64, 64]           --
│    └─MaxPool2d: 2-4                         [1, 64, 32, 32]           --
│    └─Sequential: 2-5                        [1, 64, 32, 32]           --
│    │    └─BasicBlock: 3-1                   [1, 64, 32, 32]           73,984
│    │    └─BasicBlock: 3-2                   [1, 64, 32, 32]           73,984
│    │    └─BasicBlock: 3-3                   [1, 64, 32, 32]           73,984
│    └─Sequential: 2-6                        [1, 128, 16, 16]          --
│    │    └─BasicBlock: 3-4                   [1, 128, 16, 16]          230,144